#3.-VQE

### Setup

In [ ]:
pip install qiskit

In [ ]:
pip install qiskit-nature

In [ ]:
pip install pylatexenc

In [ ]:
pip install pyscf

In [ ]:
token=""

##3.1- A simple example of VQE

In [ ]:
from qiskit_nature.drivers import Molecule
from qiskit_nature.drivers.second_quantization import ElectronicStructureMoleculeDriver, ElectronicStructureDriverType
from qiskit_nature.problems.second_quantization import ElectronicStructureProblem
from qiskit_nature.converters.second_quantization import QubitConverter
from qiskit_nature.mappers.second_quantization import JordanWignerMapper

mol = Molecule(geometry=[['H', [0., 0., -0.37]],
                              ['H', [0., 0., 0.37]]],
                     charge=0, multiplicity=1)

driver = ElectronicStructureMoleculeDriver(mol, basis='sto3g', 
        driver_type=ElectronicStructureDriverType.PYSCF)
problem = ElectronicStructureProblem(driver)
qconverter = QubitConverter(JordanWignerMapper())
secqop = problem.second_q_ops()
print(secqop[0])
qhamiltonian = qconverter.convert(secqop[0])
print("Qubit Hamiltonian")
print(qhamiltonian)

In [ ]:
from qiskit.circuit.library import EfficientSU2

ansatz = EfficientSU2(num_qubits=4, reps=1, entanglement="linear", insert_barriers = True)
ansatz.decompose().draw("mpl")

In [ ]:
from qiskit.algorithms import VQE
from qiskit import Aer
from qiskit.utils import QuantumInstance
import numpy as np
from qiskit.algorithms.optimizers import COBYLA

seed = 1234
np.random.seed(seed)

optimizer = COBYLA()

initial_point = np.random.random(ansatz.num_parameters)
quantum_instance = QuantumInstance(backend = Aer.get_backend('aer_simulator_statevector'))

vqe = VQE(
    ansatz=ansatz,
    optimizer=optimizer,
    initial_point=initial_point,
    quantum_instance=quantum_instance
)

result = vqe.compute_minimum_eigenvalue(qhamiltonian)

print(result)


In [ ]:
from qiskit.algorithms import NumPyMinimumEigensolver
solver = NumPyMinimumEigensolver()
result = solver.compute_minimum_eigenvalue(qhamiltonian)
print(result)

In [ ]:
from qiskit.algorithms import VQD
solver = VQD(ansatz=ansatz,
    optimizer=optimizer,
    initial_point=initial_point,
    quantum_instance=quantum_instance,
    k = 2)
result = solver.compute_eigenvalues(qhamiltonian)
print(result)

In [ ]:
from qiskit_nature.algorithms import GroundStateEigensolver

solver = GroundStateEigensolver(qconverter, vqe)
result = solver.solve(problem)
print(result)

In [ ]:
print(result.total_energies)

##3.2- A more complicated ansatz

In [ ]:
from qiskit_nature.algorithms import VQEUCCFactory

vqeuccf = VQEUCCFactory(quantum_instance = quantum_instance)
vqeuccf.get_solver(problem, qconverter).ansatz.decompose().draw(filename="ucc.png",output="mpl")

In [ ]:
solver = GroundStateEigensolver(qconverter, vqeuccf)
result = solver.solve(problem)
print(result)

##3.3- Noisy simulation

In [ ]:
from qiskit.providers.aer import AerSimulator
from qiskit import IBMQ

IBMQ.save_account(token)
provider = IBMQ.load_account()
backend = provider.get_backend('ibmq_manila')
quantum_instance = QuantumInstance(backend = AerSimulator.from_backend(backend),
                                   seed_simulator=seed, seed_transpiler = seed)

vqe = VQE(
    ansatz=ansatz,
    optimizer=optimizer,
    initial_point=initial_point,
    quantum_instance=quantum_instance
    #shots=1024
)

solver = GroundStateEigensolver(qconverter, vqe)
result = solver.solve(problem)
print(result)

In [ ]:
from qiskit.utils.mitigation import CompleteMeasFitter

quantum_instance = QuantumInstance(backend = AerSimulator.from_backend(backend),
                                   measurement_error_mitigation_cls=CompleteMeasFitter,
                                   seed_simulator=seed, seed_transpiler = seed)


vqe = VQE(
    ansatz=ansatz,
    optimizer=optimizer,
    initial_point=initial_point,
    quantum_instance=quantum_instance
)

solver = GroundStateEigensolver(qconverter, vqe)
result = solver.solve(problem)
print(result)

In [ ]:
from qiskit_nature.runtime import VQEClient
#backend = provider.get_backend('ibm_nairobi')
backend = provider.get_backend('ibmq_qasm_simulator')

vqe = VQEClient(
    ansatz=ansatz,
    #optimizer=optimizer, DEFAULT -> SPSA
    provider=provider,
    backend=backend,
    #shots=1024,
    initial_point = initial_point,
    measurement_error_mitigation=False
)  


solver = GroundStateEigensolver(qconverter, vqe)
result = solver.solve(problem)
print(result)

In [ ]:
vqe = VQEClient(
    ansatz=ansatz,
    #optimizer=optimizer, DEFAULT -> SPSA
    provider=provider,
    backend=backend,
    #shots=1024,
    initial_point = initial_point,
    measurement_error_mitigation=True
)  # use a complete measurement fitter for error mitigation


solver = GroundStateEigensolver(qconverter, vqe)
result = solver.solve(problem)
print(result)